# TF2.0 Loader製作- Memory

## 內容
    * 基本loader
    * cache
    * prefetch

In [1]:
import tensorflow as tf
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

**以tf.data.Dataset.from_generator這種loader類型示範**

先準備好一個data generator，會yield指定數量的數字

In [2]:
def f(rng):
    for i in range(rng):
        yield i

In [3]:
list(iter(f(3)))

[0, 1, 2]

---

## 最陽春的loader

In [4]:
d=tf.data.Dataset.from_generator(f,args=[3],output_types=tf.float32)

In [5]:
%%time
for i,x in enumerate(d):
        pprint(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>
<tf.Tensor: shape=(), dtype=float32, numpy=1.0>
<tf.Tensor: shape=(), dtype=float32, numpy=2.0>
Wall time: 310 ms


---

## Cache

In [6]:
d=tf.data.Dataset.from_generator(f,args=[1000],output_types=tf.float32).cache()

In [7]:
%%time
for i,x in enumerate(d):
    if i<3:
        pprint(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>
<tf.Tensor: shape=(), dtype=float32, numpy=1.0>
<tf.Tensor: shape=(), dtype=float32, numpy=2.0>
Wall time: 154 ms


In [8]:
%%time
'''因為第一次跑完存進記憶體，第二次會變快(但因為存起來，有隨機型preprocess的話就不隨機了)'''
for i,x in enumerate(d):
    if i<3:
        pprint(x)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>
<tf.Tensor: shape=(), dtype=float32, numpy=1.0>
<tf.Tensor: shape=(), dtype=float32, numpy=2.0>
Wall time: 62 ms


---

## Prefetch

In [23]:
'''在跑的過程中pipeline處理，一部分先放進memory內'''
d0=tf.data.Dataset.from_generator(f,args=[1000],output_types=tf.float32)
d1=tf.data.Dataset.from_generator(f,args=[1000],output_types=tf.float32).prefetch(10)
d2=tf.data.Dataset.from_generator(f,args=[1000],output_types=tf.float32).prefetch(100)
d3=tf.data.Dataset.from_generator(f,args=[1000],output_types=tf.float32).prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
%%time
for i in d0:
    pass

Wall time: 150 ms


In [25]:
%%time
'''加完快一點'''
for i in d1:
    pass

Wall time: 127 ms


In [26]:
%%time
'''加越多越快'''
for i in d2:
    pass

Wall time: 126 ms


In [28]:
%%time
'''Autotune很容易達到很快'''
for i in d3:
    pass

Wall time: 130 ms


In [16]:
'''含有cache的情況也是可以加速'''
d0=tf.data.Dataset.from_generator(f,args=[10],output_types=tf.float32).cache().repeat(100)
d1=tf.data.Dataset.from_generator(f,args=[10],output_types=tf.float32).cache().repeat(100).prefetch(10)
d2=tf.data.Dataset.from_generator(f,args=[10],output_types=tf.float32).cache().repeat(100).prefetch(100)
d3=tf.data.Dataset.from_generator(f,args=[10],output_types=tf.float32).cache().repeat(100).prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
for i in d0:
    pass
for i in d1:
    pass
for i in d2:
    pass
for i in d3:
    pass

In [18]:
%%time
for i in d0:
    pass

Wall time: 58 ms


In [19]:
%%time
'''加完快一點'''
for i in d1:
    pass

Wall time: 62 ms


In [20]:
%%time
'''加越多越快'''
for i in d2:
    pass

Wall time: 61 ms


In [21]:
%%time
'''Autotune很容易達到很快，而且記憶體也不會讓他爆'''
for i in d3:
    pass

Wall time: 63 ms


---